In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from cv2 import imread, imwrite
from ipywidgets import interact, widgets
from pathlib import Path
from os import listdir
from os.path import isfile, join

In [2]:
p = Path.home()
p = p/'OneDrive'/'Documentos'/'Python'/'Results'/'Photos'/'Riscos_umidade'

In [3]:
filenames = [f for f in listdir(p) if isfile(join(p, f))]
imgs = dict()

for filename in filenames:
    filepath = p/filename
    try:
        imgs[filepath.name] = imread(str(filepath))
    except FileNotFoundError:
        pass

In [4]:
for key, img in imgs.items():
    imwrite('src/Riscos_umidade2/{}'.format(key[:-4]+'.png'), img)

In [5]:
dfs = dict()
left_edge_px = dict()
right_edge_px = dict()
high = dict()
low = dict()
width = dict()

for key, img in imgs.items():
    dfs[key] = pd.DataFrame({'Blue': imgs[key][:,:,2].sum(axis = 1),
                       'Red': imgs[key][:,:,0].sum(axis = 1),
                       'Green': imgs[key][:,:,1].sum(axis = 1)})
    high[key] = max(dfs[key]['Red'])
    low[key] = min(dfs[key]['Red'])
    
for key in list(imgs.keys()):
    left_edge_px[key] = dfs[key][dfs[key]['Red']>=np.mean((high[key],low[key]))].index.min()
    right_edge_px[key] = dfs[key][dfs[key]['Red']>=np.mean((high[key],low[key]))].index.max()
    width[key] = 1.26576*(right_edge_px[key] - left_edge_px[key])

In [6]:
def plot_colors(key):
    fig = px.line(dfs[key])
    fig.show()

In [7]:
def plot_overlay(key):
    path = p/key
    # Create figure
    fig = go.Figure()

    # Constants
    img_width = 2048
    img_height = 1532
    scale_factor = 0.3

    # Add invisible scatter trace.
    # This trace is added to help the autoresize logic work.
    fig.add_trace(
        go.Scatter(
            x=[0, img_width * scale_factor],
            y=[0, img_height * scale_factor],
            mode="markers",
            marker_opacity=0
        )
    )

    # Configure axes
    fig.update_xaxes(
        visible=False,
        range=[0, img_width * scale_factor]
    )

    fig.update_yaxes(
        visible=False,
        range=[0, img_height * scale_factor],
        # the scaleanchor attribute ensures that the aspect ratio stays constant
        scaleanchor="x"
    )

    # Add image
    fig.add_layout_image(
        dict(
            x=0,
            sizex=img_width * scale_factor,
            y=img_height * scale_factor,
            sizey=img_height * scale_factor,
            xref="x",
            yref="y",
            opacity=1.0,
            layer="below",
            sizing="stretch",
            source='src/riscos_umidade2/{}'.format(key)[:-4]+'.png')
    )
    fig.add_hline(y = 0.3*1532-0.3*left_edge_px[key], line_color = 'red')
    fig.add_hline(y = 0.3*1532-0.3*right_edge_px[key], line_color = 'red')

    # Configure other layout
    fig.update_layout(
        width=img_width * scale_factor,
        height=img_height * scale_factor,
        margin={"l": 0, "r": 0, "t": 0, "b": 0},
    )

    # Disable the autosize on double click because it adds unwanted margins around the image
    # More detail: https://plotly.com/python/configuration-options/
    fig.show(config={'doubleClick': 'reset'})

In [8]:
interact(plot_colors, key = list(dfs.keys()))

interactive(children=(Dropdown(description='key', options=('GP4-01a.tif', 'GP4-01b.tif', 'GP4-01c.tif', 'GP4-0…

<function __main__.plot_colors(key)>

In [9]:
interact(plot_overlay, key = list(imgs.keys()))

interactive(children=(Dropdown(description='key', options=('GP4-01a.tif', 'GP4-01b.tif', 'GP4-01c.tif', 'GP4-0…

<function __main__.plot_overlay(key)>

In [10]:
width

{'GP4-01a.tif': 491.11487999999997,
 'GP4-01b.tif': 475.92576,
 'GP4-01c.tif': 482.25456,
 'GP4-02a.tif': 374.66496,
 'GP4-02b.tif': 375.93072,
 'GP4-02c.tif': 384.79104,
 'GP4-03a_ne.tif': 364.53888,
 'GP4-03b.tif': 365.80464,
 'GP4-03c.tif': 373.3992,
 'GP4-04a.tif': 278.4672,
 'GP4-04b.tif': 280.99872,
 'GP4-04c.tif': 279.73296,
 'GP4-05a.tif': 363.27312,
 'GP4-05b.tif': 363.27312,
 'GP4-05c.tif': 363.27312,
 'GP4-06a.tif': 358.21008,
 'GP4-06b.tif': 360.7416,
 'GP4-06c.tif': 353.14704,
 'GP4-07a.tif': 362.00736,
 'GP4-07b.tif': 354.4128,
 'GP4-07c.tif': 356.94432,
 'GP4-08a.tif': 431.62416,
 'GP4-08b.tif': 431.62416,
 'GP4-08c.tif': 429.09264,
 'GP4-09a.tif': 259.4808,
 'GP4-09b.tif': 259.4808,
 'GP4-09c.tif': 262.01232,
 'GP4-10a.tif': 297.4536,
 'GP4-10b.tif': 298.71936,
 'GP4-10c.tif': 301.25088,
 'GP4-11a.tif': 363.27312,
 'GP4-11b.tif': 359.47584,
 'GP4-11c.tif': 368.33616,
 'GP4-12a.tif': 231.63408,
 'GP4-12b.tif': 226.57104,
 'GP4-12c.tif': 225.30528,
 'GP4-13a.tif': 263.278

In [11]:
df = pd.DataFrame()
for key, value in width.items():
    material, ensaio = key.split('-')
    ensaio = ensaio.split('_')[0].split('.')[0]
    df = df.append(pd.DataFrame([[material, ensaio[:2],value]], columns = ['Material', 'Ensaio', 'Largura']), ignore_index = True)

In [12]:
df.to_csv('data_largura', index = False)